## Sentimenet Analysis with fastai

In [ ]:
!pip install -U fastai

In [2]:
import pandas as pd
from fastai.text.all import *

In [3]:
raw_df = pd.read_excel("./drive/MyDrive/digikala.xlsx")

In [4]:
raw_df.columns

Index(['product_id', 'product_title', 'title_en', 'user_id', 'likes',
       'dislikes', 'verification_status', 'recommend', 'title', 'comment',
       'advantages', 'disadvantages'],
      dtype='object')

In [10]:
df = raw_df.copy()
df["text"] = df["title"].astype(str) + "\r\n" + df["comment"].astype(str)

In [ ]:
df = df[df.recommend.isin(["recommended", "not_recommended"])].loc[:, ["text", "recommend"]]

### Specific text classification task with pretrained model


In [13]:
dls = TextDataLoaders.from_df(df, text_col="text", label_col="recommend", valid="test")

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [14]:
dls.show_batch()

,text,category
0,xxbos یکی از بهترین ها در رده قیمتی خودش \r\n الان 2 ماه میشه که ازین تلوزیون توی اتاقم استفاده xxunk قبل از این با تلوزیون هوشمند کار نکرده xxunk ظاهر خوبی داره و از نظر طراحی قابل قبوله ولی انتظار داشتم ضخامت تلوزیون باریکتر می بود.این تلوزیون سیستم عامل xxunk 3.5 داره که کار کردن باهاش راحته و کنترل اون مثل موس عمل میکنه ولی توی بعضی از انیمیشن های رابط کاربری کمی لگ مشاهده میشه که البته اصلا آزاردهنده xxunk برنامه های کاربردی این سیستم xxunk ما xxunk زیاد نیست ولی بین اونهایی که کاربردی به نظر xxunk میشه آپارات و فیلیمو رو نام برد که میتونید از xxunk store ال جی دانلود کنید و روی تلوزیون نصب xxunk چند نرم افزار برای کنترل تلوزیون و شیر کردن محتوای گوشی روی تلوزیون وجود داره که در صورت تمایل میتونید اونها رو روی گوشیتون نصب xxunk xxup lg xxup,recommended
1,xxbos تبلت xxunk خوش ساخت و خوش دست \r\n سلام دوستان \r\n من حدوداً یک سال و نیم پیش این تبلت رو از همینجا خریدم ، البته که اون موقع قیمتش خیلی معقول تر بود . \r\n در مورد این تبلت ، اولین چیزی که با دیدنش مجذوبش میشین ، پک عالی و با کیفیت کار هست ؛ داخل بسته بندی ، خود تبلت ، قلم تبلت با سری مخصوص کیبورد ، و حدودا 5 تا سری خودکار xxunk به همراه یک پک کاغذ مخصوص نوت برداری و نقاشی داره . \r\n این تبلت ضخامت بسیار کمی داره طوری که با برگردوندن کیبورد به پشت تبلت ، باز هم مجموع ضخامتش از یه تبلت عادی کمتره و همین خیلی باعث خوش دست بودنش میشه ؛ وزن نسبتاً کمی داره . \r\n 64 گیگ حافظه داخلی داره که طبیعتاً کمه ، من خودم از یه کارت حافظه 128 گیگی استفاده می,recommended
2,xxbos پرینتری همه کاره \r\n سلام ، این پرینتر رو از دیجی کالا خریدم ، روز اول به راحتی از روی دفترچه خودش کامل و به راحتی راه انداختمش و xxunk نصب کردم و xxunk که تو جعبه بود رو پر کردم و … ولی هر کاری کردم نتونستم با وای فای وصل شم . فکر کردم پرینتر خرابه و خیلی تو ذوقم خورد ، اما فرداش با یخورده جستجو تو اینترنت فهمیدم توضیحات دفترچه اش راجع به اتصال وای فای ناقصه ، درصورتیکه این اتصال به راحتی برقرار میشه ، الانم راه حل رو اینجا میذارم تا شما هم مثه من اذیت نشین : \r\n 1- بعد از اینکه پرینتر کامل راه افتاد و کاغذ توش گذاشتین ، دکمه i در قسمت wireless روی پرینتر رو بزنید ، با اینکار یکی دو صفحه اطلاعات خود پرینتر چاپ میشه ، اینطوری هم یه تستی از پرینتر xxunk که سالمه و,recommended
3,xxbos خیلی خوب \r\n من به ندرت کیس عوض میکنم ولی چون کیس قبلیم خیلی صدا میداد و خوب خنک نمیکرد مجبور شدم عوضش کنم . این کیس رو پس از بررسی زیاد انتخاب کردم که دیروز به دستم xxunk از همه که شاسی و xxunk فلزی بدنه خیلی محکم و راضی کنندس . دکمه پاور بزرگ و ریست کوچک خیلی خوب طراحی شدن . ظاهر کیس هم که خیلی xxunk و برا خودش ابهت خاصی داره . دو پورت xxup usb 3 و دو پورت xxup usb 2 که جلوش تعبیه شده خیلی کار راه اندازه . استندش برا هدست خیلی خوبه چون همیشه مشکل داشتم با هدست . چون بدنه کیس محکمه وزنشم نسبتا زیاده پس دستگیره واقعا لازم بوده که گذاشتن . سه تا فن 12 شیشه ای با نور سفید مارک خود گرین توش کار شده که قیمت خود فن ها کم نیست . در کل,recommended
4,xxbos ساعت هوشمند \r\n من این مدل رو از دیجیکالا گرفتم با یه استند که به همراهش بود . در پوش پشت ساعت رو که باز کنید میتونید جای ورودی سیم کارت رو مشاهده کنید . با نصب سیم کارت و شارژ کامل ساعت نور صفحه ساعت هم از متوسط بیشتر گذاشتم و سنسور xxunk رو هم فعال کردم که با بالا آوردن مچ صفحه ساعت روشن بشه ولی با این حال باطری این ساعت xxunk شارژ به شما میده و بعد خاموش میشه البته بعد یه مدت استفاده بدتر هم میشه برای من الان به زور 6ساعت شارژ نگه میداره . واچ فیس ها هم محدوده نه xxunk داره که دانلود کنی و نه میتونید نرم افزار روی ساعت بریزی . کیفیت نمایش صفحه ساعت بد نیست واز متوسط به بالا هست . کیفیت دور بین هم بد نیست در حد متوسط و هم قابلیت فیلم xxunk داره و,not_recommended
5,xxbos کیفیت بالا خدمات خوب \r\n من نمونه آب خروجی این دستگاه و نمونه آب شیر رو فرستادم آزمایشگاه ، نتیجه اینکه شاخص املاح آب از 200 به 2 xxunk پیدا کرده بود که نشون میده دستگاه کارش رو درست انجام میده . ولی لزوما به این معنی نیست که نتیجه خوب هست . \r\n در حال حاضر نگرانی من برای استفاده از آب در پخت و xxunk ، دستگاه چای ساز ، کتری ، دستگاه قهوه ساز و یخچال برطرف شده و املاح زیاد آب به این دستگاه ها آسیب نمیزنه اما این آب مناسب نوشیدن نیست و به گفته کارشناس آزمایشگاه برای اینکه آب سالمی مصرف بشه نیا

In [15]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [16]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.616087,0.618082,0.694612,01:07


epoch,train_loss,valid_loss,accuracy,time
0,0.620028,0.617701,0.694612,01:36
1,0.612634,0.615339,0.694612,01:36
2,0.618190,0.612532,0.694612,01:37
3,0.610114,0.614850,0.694235,01:37


In [17]:
learn.show_results()

,text,category,category_
0,"xxbos بررسی sandisk xxmaj ultra xxmaj fit xxmaj flash 128 gb \r\n می بخشید طولانیه اما امیدوارم مفید باشه براتون . \r\n قبل هر چیزی در مورد نظرات یه مطلب بگم ، اینکه بعضی دوستان گفتن خوبه و سرعتش بالاست و بعضی دوستان هم گفتن بد و خیلی سرعتش پایین … در هر دو حالات هر xxunk درست گفتن ! نکته اینجاست ، کسی که میاد و نظرش میگه مشخصات سخت افزاری xxunk ( لب تاپ یا کیس و .. ) رو نمیگه که خیلی خیلی مهمه ! مثلا سرعت خواندن و نوشتن هاردش 5400 دوره یا xxunk دوره ، ( البته سرعت یک هارد xxunk فیک ممکنه از 5400 اصل کمتر باشه ) یا میزان رم و xxup xxunk سیستم و بافر آن یا حتی xxup cpu که در مچ بودن سیستم و انتقال سرعت بسیار مهمه و از همه مهمتر نوع پورت xxup usb سیستم و جنس """,recommended,recommended
1,xxbos هدفون بلاژیو xxmaj faith2 \r\n با سلام \r\n بنده این محصول رو بعد از دو هفته تحقیق در مورد تمامی هدفون ها و امتحان کردن اکثر مارک های معروف و غیر معروف ار دیجی کالا خریداری کردم و حتی قبل از اینکه به دستم برسه صد در صد از کیفیت محصولی که خرید کردم مطمئن بودم بیشتر از این نیاز به تعریف این محصول نیست فقط چند نکته بسیار مهم در مورد تفاوت اساسی این هدفون با هدفون های دیگه خدمت دوستان عرض میکنم تا تونسته باشم در انتخاب محصولی که میخوان خرید کنند راهنمایی موثری کرده باشم . \r\n اولین دلیل خوبی این هدفون کیفیت ساخت و ارگونومی خوبشه که در این شکی نیست اما دلایل مهمتر از این قراره که این هدفون چند مورد بسیار خاص داره یکی اینکه پاسخ فرکانسی این هدفون از 15 تا 25 هزار کیلو هرتز هست این یعنی 5 هرتز پایین تر,recommended,recommended
2,xxbos دیگه تولید نمیشه وگرنه ارزون ترین و با امکانات ترین xxunk xxunk که دیدم . بخرید \r\n اگه دستگاه xxunk ( مثلا موبایلتون ) ۵ گیگا هرتز رو ساپورت میکنه و تو محیط کوچیک میخوایید استفاده کنید ، حتما بگیرید . \r\n در غیر این صورت هزینه بیخوده . \r\n\r\n اول بگم که کلا تحقیقی نداریم که بتونه ثابت کنه امواج وایفای مودم ، برای انسان مضره ! چون معمولا بیشتر از یک متر با آدم فاصله داره . چون اسیب این امواج با توان دوم فاصله رابطه عکس داره و با توجه به توان بسیار پایین امواج ، در فواصل بالای یک متر اثر اونها از اثر امواج موتور جارو برقی کمتره xxrep 5 ! xxunk \r\n\r\n اما با این وجود ، فرکانس ۵ گیگا هرتز هم سرعت بالاتری رو ساپورت میکنه و هم اسیب کمتری ( قدرت نفوذ کمتری ) داره .. یعنی زورش کمتر میرسه که,recommended,recommended
3,xxbos تجربه استفاده یکماهه \r\n با سلام \r\n بنده این دستگاه رو از دیجی کالا خریدم و تو پیشنهاد ویژه . \r\n تقریبا میشه گفت دستگاه خوبیه و حس جالبی به شما میده xxunk اگه به سینمای xxunk بشه واقعا حس داشتن یک سینمای واقعی رو به شما میده / اما چند تا مشکل داره که من xxunk میگم / \r\n اول اینکه کیفیت خروجی صدای دستگاه پایینه xxup aux xxup out داره اما در حد یه گیرنده دیجیتال چینی به شما کیفیت و بلندی صدا میده و شما هر سیستم صوتی به دستگاه وصل کنید انتظار صدای فوق العاده زیاد رو نداشته باشید اما اگر سیستم خوبی دارید و ساب ووفر داره میتونید باهاش فیلم ببینید اما دیدن کلیپ ویدیویی رو اصلا پیشنهاد نمیکنم و اینکه مثل موبایل خروجی بلوتوث نداره که بشه باهاش به باند بلوتوث وصل شد و حتما برای اتصال باند قوی تر باید از طریق,recommended,recommended
4,xxbos بررسی هدفون \r\n مشخصا این هدفون با تناسب قیمتی که برای آن وجود دارد در این xxunk یعنی هدفون بیسیم بسیار مناسب است . \r\n قرارگیری در گوش کاملا با xxunk گوش سازگار است \r\n در پیک صدا صدای خیلی بالائی ندارد و اگر بخواهیم این موضوع را مهندسی تر بررسی کنیم در تمام هدفون ها وقتی قصد xxunk صدای هدفون را دارید xxunk مبنی بر آزار گوش در صدای بلند روی صفحه موبایل ضاهر میشود که حد متناسب گوش کردن موزیک را به شما اعلان میکند و این هدفون دقیقا همون حد را بعنوان بالاترین مدنظر قرار داده که برای سلامت گوش ضرری در پی نخواهد داشت و در واقع با بالاترین حد صدا گوش به هیچ عنوان در خطر نخواهد بود . \r\n گاها دوستان از xxmaj pair نشدن هر دو گوشی نارضایتی داشته اند که تجربه شخصی به جهت کارائی و راندمان استفاده به حضور اعلام,recommended,recommended
5,xxbos عدم کارایی با اطلاعات ذکر شده در سایت \r\n علیرغم اعلام در سایت دیجی ، میزان اکسیژن خون را نشان نمی دهد . بعد از مطالعه بروشور و مشخصات ذکر شده بر روی جعبه کالا ، اصلا این قابلیت در ساعت وجود ندارد . لطفا اطلاعات اشتباه در سایت قید نشود . \r\n لطفا راه ارتباط با فروشنده را قرار دهید تا امکان سوال از فروشنده 

In [28]:
learn.predict("خیلی عالی بود")

('recommended', tensor(1), tensor([0.3705, 0.6295]))

### Fine-Tune pretrained Language Model on our data

In [29]:
dls_lm = TextDataLoaders.from_df(df, text_col="text", label_col="recommend", is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=5)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos خیلی خوبه ، میشه گفت یه غول پر قدرت با چشم های معمولی \r\n اول اینکه این گوشی برای \r\n بازی \r\n فیلم \r\n نت \r\n برنامه نویسی \r\n عالیه و دوم اینکه با وجود xxunk برنامه کمکی که در خود رام به صورت xxunk وجود داره از نصب ده ها برنامه ( پاک کردن کش ویرایش عکس و فیلم و … ) خلاص میشید ، برای استفاده بهتر از قسمت,خیلی خوبه ، میشه گفت یه غول پر قدرت با چشم های معمولی \r\n اول اینکه این گوشی برای \r\n بازی \r\n فیلم \r\n نت \r\n برنامه نویسی \r\n عالیه و دوم اینکه با وجود xxunk برنامه کمکی که در خود رام به صورت xxunk وجود داره از نصب ده ها برنامه ( پاک کردن کش ویرایش عکس و فیلم و … ) خلاص میشید ، برای استفاده بهتر از قسمت ریموت
1,\r\n باتری و دوام اون واقعا عالیه ، یه نمه سیم هندزفری ها نازکه ولی اصلا نگران نباشید چون با کیفیت و چیزیش نمیشه ( به شرطی که باهاش کشتی نگیرید ! ) \r\n یکی دیگه از خوبیای هندزفری بلوتوث های xxup lg اینه که مثل اکثر شرکتا سیم هندزفری الکی xxunk نیست که تو دست و پا باشه ، یا با اهنربا متصل میشه به بدنه دستگاه یا سیم جمع کن,باتری و دوام اون واقعا عالیه ، یه نمه سیم هندزفری ها نازکه ولی اصلا نگران نباشید چون با کیفیت و چیزیش نمیشه ( به شرطی که باهاش کشتی نگیرید ! ) \r\n یکی دیگه از خوبیای هندزفری بلوتوث های xxup lg اینه که مثل اکثر شرکتا سیم هندزفری الکی xxunk نیست که تو دست و پا باشه ، یا با اهنربا متصل میشه به بدنه دستگاه یا سیم جمع کن داره
2,\r\n وصل کردن به کامپیوتر و لپ تاپ از طریق lan \r\n قابلیت استفاده از میکرو اس دی از طریق مرورگر \r\n خاموش بودن پیش فرض wps \r\n ۲۳ ساعت باطری برای استفاده مدام از 3 g \r\n گارانتی ۳۶ ماه تعویض xxunk ( نمایندگی اصلی xxunk در ایران ) \r\n سرعت استفاده در حال 3 g دو مگابایت دانلود و ۵۰۰ کیلوبایت آپلود \r\n داغ کردن خیلی خفیف در موقع استفاده,وصل کردن به کامپیوتر و لپ تاپ از طریق lan \r\n قابلیت استفاده از میکرو اس دی از طریق مرورگر \r\n خاموش بودن پیش فرض wps \r\n ۲۳ ساعت باطری برای استفاده مدام از 3 g \r\n گارانتی ۳۶ ماه تعویض xxunk ( نمایندگی اصلی xxunk در ایران ) \r\n سرعت استفاده در حال 3 g دو مگابایت دانلود و ۵۰۰ کیلوبایت آپلود \r\n داغ کردن خیلی خفیف در موقع استفاده \r\n
3,ابتدا بوی الکل و صابون میده با کمی گذشتن بوی الکل از بین میرود و بوی صابون می ماند از خریدم پشیمانم آبی رنگش بوی بهتری دارد xxbos دقیقا مطابق با تصاویر و دارای بارکد \r\n عالیه و با کیفیت و سرعتش هم خوبه \r\n من از فروشنده xxunk خریده بودم xxbos عالی و بی نظیر \r\n در یک کلام عالیه و فراتر از انتطار شماست چون عملکردش فوق العاده خوبه \r\n,بوی الکل و صابون میده با کمی گذشتن بوی الکل از بین میرود و بوی صابون می ماند از خریدم پشیمانم آبی رنگش بوی بهتری دارد xxbos دقیقا مطابق با تصاویر و دارای بارکد \r\n عالیه و با کیفیت و سرعتش هم خوبه \r\n من از فروشنده xxunk خریده بودم xxbos عالی و بی نظیر \r\n در یک کلام عالیه و فراتر از انتطار شماست چون عملکردش فوق العاده خوبه \r\n به
4,من پاور های زیادی مثل روموس . انرجایزر . سیلیکون پاور . شیائومی رو استفاده کردم و به جرات میتونم بگم انکر توی همشون سر بوده چه از نظر کیفیت و چه از نظر شارژ شدن و شارژ کردن دستگاه . سایز مناسبی داره و وزنش هم نسبت به xxunk نرمال و مناسبه . انکر چون تخصصی توی بخش سیو انرژی کار میکنه همیشه تو این بخش حرف داشته واسه گفتن .,پاور های زیادی مثل روموس . انرجایزر . سیلیکون پاور . شیائومی رو استفاده کردم و به جرات میتونم بگم انکر توی همشون سر بوده چه از نظر کیفیت و چه از نظر شارژ شدن و شارژ کردن دستگاه . سایز مناسبی داره و وزنش هم نسبت به xxunk نرمال و مناسبه . انکر چون تخصصی توی بخش سیو انرژی کار میکنه همیشه تو این بخش حرف داشته واسه گفتن . این


In [31]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [32]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.868566,5.679263,0.141222,292.733490,01:31


In [35]:
# fit last layer
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.865128,4.711294,0.221739,111.195976,01:33


In [36]:
# fit hidden layers
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.650150,4.621910,0.228286,101.688034,01:33
1,4.532356,4.579590,0.232649,97.474449,01:32


In [37]:
TEXT = "گوشی خوبی هست ولی"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

گوشی خوبی هست ولی خوب نیست 
 اصلا جالب نیست اصلا ب درد نمی خوره کیفیت بالا 
 این هدفون رو تازه خریدم و ازش استفاده میکنم ، بسیار عالیست و با اینکه باهاش کار کردم از همه لحاظ عالیه و وزن کمی
گوشی خوبی هست ولی من چند ساله خریدم فکر کردم 
 همون اول که گوشی رو با آیفون وصلش کردم هیچ مشکلی نداشته کیفیت محصول 
 سلام دوستان 
 خیلی زیباست 
 شیک و مقاوم 
 ظاهر قشنگی داره 
 نوک دکمه ها


In [38]:
learn.save_encoder('fine_tuned')

### Use new language model for text classification task

In [39]:
dls_clas = TextDataLoaders.from_df(df, text_col="text", 
                                   label_col="recommend", 
                                   valid="test", 
                                   text_vocab=dls_lm.vocab)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [40]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [41]:
learn = learn.load_encoder('fine_tuned')

In [42]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.199918,0.164505,0.936982,01:08


In [43]:
# unfreeze all layers except 2 last layers
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.168610,0.132024,0.949322,01:11


In [46]:
learn.predict("خیلی بد بود")

('not_recommended', tensor(0), tensor([0.9619, 0.0381]))

In [47]:
learn.predict("افتضاح بود")

('not_recommended', tensor(0), tensor([0.9553, 0.0447]))

In [48]:
learn.predict("عالی بود")

('recommended', tensor(1), tensor([0.0093, 0.9907]))